In [47]:
import pandas as pd
import numpy as np

In [48]:
df = pd.read_csv("nba.csv")

In [ ]:
# Change "price obj" to int
# All glory to Mr OpenAI for this line of code
df["2025-26"] = (
    df["2025-26"]
      .astype("string")
      .str.replace(r"[\$,]", "", regex=True)  # remove $ and commas
      .str.strip()
      .pipe(pd.to_numeric, errors="coerce")
      .astype("Int64")                        # keeps missing as <NA>
)

# Change player data type from obj to str 
df["Player"] = df["Player"].astype(str)

# Drop all NA in the 2025-26 col
df = df.dropna(subset=["2025-26"])

# Rename player id col
df = df.rename(columns={"-9999": "player_id"})

# Only keep for 2025-25 <- this will be the test
df = df.drop(columns=["Rk", "2026-27","2027-28","2028-29","2029-30" ,"2030-31"	,"Guaranteed"])
df

,Player,Tm,2025-26,player_id
0,Stephen Curry,GSW,59606817,curryst01
1,Joel Embiid,PHI,55224526,embiijo01
2,Nikola Jokić,DEN,55224526,jokicni01
3,Kevin Durant,HOU,54708609,duranke01
4,Jayson Tatum,BOS,54126450,tatumja01
...,...,...,...,...
464,Didi Louzada,POR,268032,louzama01
465,Mac McClung,IND,164060,mccluma01
466,Garrison Mathews,IND,131970,mathega01
467,Garrison Mathews,IND,131970,mathega01


In [50]:
# Player Can be listed twice ?!??!
# Different teams is one reason 
# Jeremiah Robinson-Earl tho ^^^ See above

vc = df["Player"].value_counts()
df[df["Player"].map(vc) > 1].sort_values("Player")

,Player,Tm,2025-26,player_id
372,Blake Wesley,POR,7022602,weslebl01
275,Blake Wesley,WAS,7022602,weslebl01
9,Bradley Beal,PHO,59020270,bealbr01
258,Bradley Beal,LAC,59020270,bealbr01
383,Cole Anthony,MIL,15396274,anthoco01
141,Cole Anthony,MEM,15396274,anthoco01
8,Damian Lillard,MIL,68230450,lillada01
127,Damian Lillard,POR,68230450,lillada01
203,Deandre Ayton,LAL,43654814,aytonde01
44,Deandre Ayton,POR,43654814,aytonde01


In [51]:
df = df.drop_duplicates(subset=["player_id"], keep="first")
df["Player"].value_counts()

Player
Jaden Springer        1
Stephen Curry         1
Joel Embiid           1
Nikola Jokić          1
Kevin Durant          1
                     ..
Paul George           1
LeBron James          1
Karl-Anthony Towns    1
Devin Booker          1
Jaylen Brown          1
Name: count, Length: 458, dtype: int64

In [52]:
df[["Tm", "2025-26"]].groupby("Tm").agg("mean").sort_values("2025-26",ascending=False)   

,2025-26
Tm,
NYK,14917737.714286
GSW,14667247.642857
LAC,14562540.307692
MIN,14470069.857143
CLE,14445677.1875
LAL,14198677.307692
ORL,13819373.714286
HOU,13665712.333333
BOS,13483481.6


In [53]:
pay = df["2025-26"].dropna()    
print(f"Total Salries for 2025-26: ${pay.sum():,.2f}. For {len(pay)} players, For an average salary of ${ pay.sum() / len(pay):,.2f}") 

Total Salries for 2025-26: $5,697,186,886.00. For 458 players, For an average salary of $12,439,272.68
